### Dependecies and necessary tools

In [1]:
#Importing dependencies
import pandas as pd
from sodapy import Socrata
import scipy.stats as st
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

# Importing data for API call
from config1 import app_token
from config1 import username
from config1 import password

### EXTRACT 

In [30]:
# Getting the data from API NY Open Data for the first database: 
client = Socrata("data.cityofnewyork.us",
                  app_token, username,
                  password)
# Database returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("gaq9-z3hz", limit=2832)

# Convert to pandas DataFrame
Recycling_Div_Captures = pd.DataFrame.from_records(results)
Recycling_Div_Captures

,_zone,district,fiscal_month_number,fiscal_year,month_name,diversion_rate_total_total_recycling_total_waste_,capture_rate_paper_total_paper_max_paper_,capture_rate_mgp_total_mgp_max_mgp_,capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100
0,Brooklyn North,BKN01,10,2019,April,14.6870926033314,44.9091597758182,43.0340618265488,44.1467643388238
1,Brooklyn North,BKN02,10,2019,April,19.9501814568266,34.1940201111571,57.9470313498761,41.2136999269730
2,Brooklyn North,BKN03,10,2019,April,12.1641613296565,33.5215567195417,44.9197311047560,38.1559365633997
3,Brooklyn North,BKN04,10,2019,April,15.5418031271589,35.2113607057133,68.5112600518364,48.7507554470644
4,Brooklyn North,BKN05,10,2019,April,10.0518454148520,22.2654301541493,45.0517913754661,31.5301290076712
...,...,...,...,...,...,...,...,...,...
2827,Queens West,QW06,3,2016,September,20.1214735183925,30.4196893217642,67.9561359185218,39.0333079418224
2828,Queens West,QW09,3,2016,September,17.4348618222880,41.1065470054454,79.6950721141275,54.2727362237205
2829,Staten Island,SI01,3,2016,September,18.6838350777916,39.5199429139324,71.7164916919349,49.6526689393673
2830,Staten Island,SI02,3,2016,September,19.0168187778538,44.4579262409406,74.9601995452711,54.0574389302287


In [3]:
# creating path for the citywide subsort 
data_path = "Resources/Citywide_Subsort.csv"


# Read the data into df
cws_df = pd.read_csv(data_path)


#Preview data
cws_df.head()

,Material,Aggregate Percent,Refuse Percent,MGP Percent,Paper Percent,Organic Percent,Material Group,DSNY Diversion Summary Category,Location
0,Newspaper,0.019,0.011,0.005,0.098,0.000,Paper,Designated Paper,Citywide
1,Plain OCC/Kraft Paper,0.056,0.012,0.010,0.464,0.002,Paper,Designated Paper,Citywide
2,Paper Bags: Kraft Grocery,0.002,0.001,0.000,0.008,0.000,Paper,Designated Paper,Citywide
3,Paper Bags: Non-food Retail,0.002,0.002,0.000,0.008,0.000,Paper,Designated Paper,Citywide
4,Paper Bags: Fast Food Bags,0.001,0.002,0.000,0.001,0.000,Paper,Designated Paper,Citywide


In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\da34s\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [5]:
url = 'https://data.cityofnewyork.us/City-Government/DSNY-Waste-Characterization-Mainsort/k3ks-jzek'
browser.visit(url)

In [6]:
df_list = []
for x in range(1,41):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    table = soup.find_all('div', class_='socrata-table frozen-columns')
    df_list.append(pd.read_html(str(table))[0])
    button = browser.find_by_xpath('//*[@id="app"]/div/div[2]/section[4]/div[2]/div/div/div[4]/div[3]/span[1]/button[2]').click()

In [7]:
df_combined = pd.concat(df_list)

In [8]:
df_combined.reset_index(drop=True, inplace=True)

In [14]:
cwm_df = df_combined
cwm_df.head()

,Material,Aggregate Percent,Refuse Percent,MGP Percent,Paper Percent,Organic Percent,Material Group,DSNY Diversion Summary Category,Location
0,Newspaper,1.9%,1.1%,0.5%,9.8%,0%,Paper,Designated Paper,Citywide
1,Plain OCC/Kraft Paper,5.7%,1.4%,1%,47.2%,0.2%,Paper,Designated Paper,Citywide
2,High Grade Paper,0.7%,0.5%,0.1%,2.6%,0%,Paper,Designated Paper,Citywide
3,Mixed Low Grade Paper,8.7%,6.8%,2.4%,31.4%,0.9%,Paper,Designated Paper,Citywide
4,Paper: Compostable/Soiled/Waxed OCC/Kraft,8.1%,9.8%,0.7%,1.4%,2%,Paper,Organics Suitable for Composting,Citywide


### TRANSFORM

In [25]:
# Analizing material groups
unique_mat_subsort = cws_df["Material Group"].unique()
print(f'Material groups (subsort): {unique_mat_subsort}')
unique_mat_mainsort = cwm_df["Material Group"].unique()
print(f'Material groups (mainsort): {unique_mat_mainsort}')

Material groups (subsort): ['Paper' 'Plastic' 'Glass' 'Metal' 'Organic' 'E-Waste' 'C&D'
 'Special Waste' 'Miscellaneous Inorganics']
Material groups (mainsort): ['Paper' 'Plastic' 'Glass' 'Metal' 'Organic' 'E-Waste' 'C&D'
 'Special Waste' 'Miscellaneous Inorganics']


In the general database (Recycling_Div_Captures) is possible to see that there is just data for paper and MGP, so from this dataset information just for those materials will be taken. 
The materials will be group in the next collections for a non-relational dabatase:
- Paper
- MGP (Metal, Glass, and Plastic)
- Organic
- E- Waste
- Special Waste
- Others

In [24]:
unique_loc_subsort = cws_df["Location"].unique()
print(f"Locations register in the Subsort dataset: {unique_loc_subsort}")
print("----------------------------------------------------------------")
unique_loc_mainsort = cwm_df["Location"].unique()
print(f"Locations register in the Mainsort dataset: {unique_loc_mainsort}")
print("----------------------------------------------------------------")
uniqie_loc_general = Recycling_Div_Captures["_zone"].unique()
print(f"Locations register in the general dataset: {uniqie_loc_general}")

Locations register in the Subsort dataset: ['Citywide' 'Brooklyn' 'Queens' 'Manhattan' 'Staten Island' 'Bronx'
 'Schools' 'NYCHA']
----------------------------------------------------------------
Locations register in the Mainsort dataset: ['Citywide' 'Brooklyn' 'Queens' 'Manhattan' 'Staten Island' 'Bronx'
 'Schools' 'NYCHA']
----------------------------------------------------------------
Locations register in the general dataset: ['Brooklyn North' 'Brooklyn South' 'Bronx' 'Manhattan' 'Queens East'
 'Queens West' 'Staten Island']


Analaizing the locations in all the databases, we decide to summarize and filtering them and their data just for the 5 boroughs.
- Manhattan
- Queens (which includes 'Queens East' and 'Queens West')
- Brooklyn (which includes 'Brooklyn North' and 'Brooklyn South')
- Bronx
- Staten Island
Each material collection will have some next fields per location (materials like paper and MGP will have average capture rates taken from the general dataset, but the other not because we do not have that information):
- Total aggregate percent
- Total refuse percent
- Average capture rate
- Average misclassified material 


In [ ]:
# Mainsort and subsort percentages format modification 
main_cols = ["Aggregate Percent", "Refuse Percent"]


In [35]:
# First filter: Recycling_Div_Captures data should be filter just for 2017 year
Recycling_Div_Captures["fiscal_year"] = Recycling_Div_Captures["fiscal_year"].apply(pd.to_numeric, errors='coerce')
Recycling_Div_Captures = Recycling_Div_Captures.loc[Recycling_Div_Captures["fiscal_year"]==2017,:]


,_zone,district,fiscal_month_number,fiscal_year,month_name,diversion_rate_total_total_recycling_total_waste_,capture_rate_paper_total_paper_max_paper_,capture_rate_mgp_total_mgp_max_mgp_,capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100
1416,Brooklyn North,BKN01,10,2017,April,13.8011187636531,42.3090372160212,42.2502943040951,42.2851529554546
1417,Brooklyn North,BKN02,10,2017,April,17.3846057078494,33.5014979806190,54.1659431601718,39.6084200577444
1418,Brooklyn North,BKN03,10,2017,April,11.3176393729911,31.6571287740573,41.1100986323700,35.5006085713826
1419,Brooklyn North,BKN04,10,2017,April,14.3373064604498,31.4839593042329,64.6589074509537,44.9725501799468
1420,Brooklyn North,BKN05,10,2017,April,10.0387442532265,23.3716187740202,43.3362562619179,31.4890339351592
...,...,...,...,...,...,...,...,...,...
2119,Queens West,QW06,3,2017,September,21.0042803687275,30.5547127644004,74.9656739587621,40.7458500978815
2120,Queens West,QW09,3,2017,September,18.8381247380501,42.2830043367855,85.1609170781304,56.9127086555500
2121,Staten Island,SI01,3,2017,September,19.0574966444757,38.3623014491355,79.7568402401434,51.3897698095049
2122,Staten Island,SI02,3,2017,September,19.2481535186128,42.7860572186745,78.7332103228743,54.0991534238204
